# Vector Database Basics

Vector databases help us store, manage, and query the embeddings we created for generative AI, recommenders, and search engines.

Across many of the common use cases, users often find that they need to manage more than just vectors.
To make it easier for practitioners, vector databases should store and manage all of the data they need:
- embedding vectors
- categorical metadata
- numerical metadata
- timeseries metadata
- text / pdf / images / video / point clouds

And support a wide range of query workloads:
- Vector search (may require ANN-index)
- Keyword search (requires full text search index)
- SQL (for filtering)

For this exercise we'll use LanceDB since it's open source and easy to setup

In [39]:
# requirements - lancedb pandas pydantic

## Creating tables and adding data

Let's create a LanceDB table called `cats_and_dogs` under the local database directory `~/.lancedb`.
This table should have 4 fields:
- the embedding vector
- a string field indicating the species (either "cat" or "dog")
- the breed
- average weight in pounds

We're going to use pydantic to make this easier. First let's create a pydantic model with those fields

In [40]:
from lancedb.pydantic import vector, LanceModel
import numpy as np

class CatsAndDogs(LanceModel):
    vector: vector(2)
    species: str
    breed: str
    weight: float

Now connect to a local db at ~/.lancedb and create an empty LanceDB table called "cats_and_dogs"

In [41]:
import lancedb

db = lancedb.connect("~/.lancedb")

table_name = 'cats_and_dogs'
db.drop_table(table_name, ignore_missing= True)
table = db.create_table(table_name, schema=CatsAndDogs)


Let's add some data

First some cats

In [42]:
data = [
    CatsAndDogs(
        vector=[1., 0.],
        species="cat",
        breed="shorthair",
        weight=12.,
    ),
    CatsAndDogs(
        vector=[-1., 0.],
        species="cat",
        breed="himalayan",
        weight=9.5,
    ),
]

In [43]:
[dict(d) for d in data]

[{'vector': FixedSizeList(dim=2),
  'species': 'cat',
  'breed': 'shorthair',
  'weight': 12.0},
 {'vector': FixedSizeList(dim=2),
  'species': 'cat',
  'breed': 'himalayan',
  'weight': 9.5}]

Now call the `LanceTable.add` API to insert these two records into the table

In [44]:
table.add([dict(d) for d in data])

Let's preview the data

In [45]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


Now let's add some dogs

In [46]:
data = [
    CatsAndDogs(
        vector=[0., 10.],
        species="dog",
        breed="samoyed",
        weight=47.5,
    ),
    CatsAndDogs(
        vector=[0, -1.],
        species="dog",
        breed="corgi",
        weight=26.,
    )
]

In [47]:
table.add([dict(d) for d in data])

In [48]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5
2,"[0.0, 10.0]",dog,samoyed,47.5
3,"[0.0, -1.0]",dog,corgi,26.0


## Querying tables

Vector databases allow us to retrieve data for generative AI applications. Let's see how that's done.

Let's say we have a new animal that has embedding [10.5, 10.], what would you expect the most similar animal will be?
Can you use the table we created above to answer the question?

**HINT**  `search` API for LanceTable and `limit` / `to_df` APIs. For examples you can refer to [LanceDB documentation](https://lancedb.github.io/lancedb/basic/#how-to-search-for-approximate-nearest-neighbors).

In [49]:
table.search([10.5, 10.]) \
    .limit(1).to_pandas()

,vector,species,breed,weight,_distance
0,"[0.0, 10.0]",dog,samoyed,47.5,110.25


Now what if we use cosine distance instead? Would you expect that we get the same answer? Why or why not?

Add a call to `metric` in the call chain

In [50]:
table.search([10.5, 10.]).metric("cosine") \
    .limit(1).to_pandas()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,0.275862


## Filtering tables

In practice, we often need to specify more than just a search vector for good quality retrieval. Oftentimes we need to filter the metadata as well.

Please write code to retrieve two most similar examples to the embedding [10.5, 10.] but only show the results that is a cat.

In [51]:
table.search([10.5, 10.]).metric("cosine") \
    .limit(2).where("species = 'cat' ").to_pandas()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,0.275862


## Creating ANN indices

For larger tables (e.g., >1M rows), searching through all of the vectors becomes quite slow. Here is where the Approximate Nearest Neighbor (ANN) index comes into play. While there are many different ANN indexing algorithms, they all have the same purpose - to drastically limit the search space as much as possible while losing as little accuracy as possible

For this problem we will create an ANN index on a LanceDB table and see how that impacts performance

### First let's create some data

Given the constraints of the classroom workspace, we'll complete this exercise by creating 100,000 vectors with 16D in a new table. Here the embedding values don't matter, so we simply generate random embeddings as a 2D numpy array. We then use the vec_to_table function to convert that in to an Arrow table, which can then be added to the table.

In [52]:
from lance.vector import vec_to_table
import numpy as np

mat = np.random.randn(100_000, 16)
table_name = "table_ann"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, vec_to_table(mat))

In [55]:
mat[0]

array([ 1.02005811, -0.4457279 ,  0.63070853,  0.74986925, -0.89312832,
        0.7244108 ,  1.22876908, -0.03572682,  0.66263573, -0.91222494,
        0.17482974, -0.42122546,  0.2682422 ,  0.05677254, -0.1221359 ,
       -0.83425487])

### Let's establish a baseline without an index

Before we create the index, let's make sure know what we need to compare against.

We'll generate a random query vector and record it's value in the `query` variable so we can use the same query vector with and without the ANN index.

In [54]:
query = np.random.randn(16)
table.search(query).limit(10).to_pandas()

,vector,_distance
0,"[-1.8217257, -0.4105151, 0.25570652, 1.2815511...",4.564942
1,"[-1.3787163, 0.18927933, 0.8893695, 0.45555136...",4.879690
2,"[-2.1621974, -0.766172, 0.608445, 0.11594374, ...",6.521850
3,"[-1.0335839, 0.18607399, 0.44425666, -0.516900...",6.933640
4,"[-0.6950925, 0.55010796, -0.16022417, -0.44099...",6.960024
5,"[-1.7734425, -0.83179057, 0.004681533, 0.31210...",7.486131
6,"[-1.347302, -0.4239581, 0.49599501, -0.2163492...",7.536533
7,"[-0.61555696, -0.51789796, 0.6504343, 1.280870...",7.675541
8,"[-0.8363744, 0.044845145, 0.32828158, -0.00031...",7.726789
9,"[-1.1497291, -0.7359861, 1.2386738, 0.5515901,...",8.075428


Please write code to compute the average latency of this query

In [56]:
%timeit table.search(query).limit(10).to_pandas()

7.59 ms ± 756 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Now let's create an index

There are many possible index types ranging from hash based to tree based to partition based to graph based.
For this task, we'll create an IVFPQ index (partition-based index with product quantization compression) using LanceDB.

Please create an IVFPQ index on the LanceDB table such that each partition is 4000 rows and each PQ subvector is 8D.

**HINT** 
1. Total vectors / number of partitions = number of vectors in each partition
2. Total dimensions / number of subvectors = number of dimensions in each subvector
3. This step can take about 7-10 minutes to process and execute in the classroom workspace.

In [57]:
table.create_index(num_partitions= 256, num_sub_vectors=16)

Now let's search through the data again. Notice how the answers now appear different.
This is because an ANN index is always a tradeoff between latency and accuracy.

In [58]:
table.search(query).limit(10).to_pandas()

,vector,_distance
0,"[-1.8217257, -0.4105151, 0.25570652, 1.2815511...",4.554533
1,"[-1.3787163, 0.18927933, 0.8893695, 0.45555136...",4.906518
2,"[-2.1621974, -0.766172, 0.608445, 0.11594374, ...",6.418826
3,"[-1.0335839, 0.18607399, 0.44425666, -0.516900...",6.954990
4,"[-0.6950925, 0.55010796, -0.16022417, -0.44099...",6.959525
5,"[-1.7734425, -0.83179057, 0.004681533, 0.31210...",7.459003
6,"[-1.347302, -0.4239581, 0.49599501, -0.2163492...",7.516437
7,"[-0.8363744, 0.044845145, 0.32828158, -0.00031...",7.715341
8,"[-1.4424235, -1.4810574, -0.38307098, 1.238502...",8.130560
9,"[-0.44408143, -0.47051442, 0.30431873, -0.1514...",8.270196


Now write code to compute the average latency for querying the same table using the ANN index.

**SOLUTION** The index is implementation detail, so it should just be running the same code as above. You should see almost an order of magnitude speed-up. On larger datasets, this performance difference should be even more pronounced.

In [59]:
%timeit table.search(query).limit(10).to_pandas()

3.07 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Deleting rows

Like with other kinds of databases, you should be able to remove rows from the table.
Let's go back to our tables of cats and dogs

In [60]:
table = db["cats_and_dogs"]

In [63]:
len(table)

4

In [64]:
table.to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5
2,"[0.0, 10.0]",dog,samoyed,47.5
3,"[0.0, -1.0]",dog,corgi,26.0


,vector,species,breed,weight
0,"[0.0, 10.0]",dog,samoyed,47.5
1,"[0.0, -1.0]",dog,corgi,26.0


Can you use the `delete` API to remove all of the cats from the table?

**HINT** use a SQL like filter string to specify which rows to delete from the table

In [ ]:
table.delete("species = 'cat' ")

In [73]:
table.to_pandas()

,vector,species,breed,weight
0,"[0.0, 10.0]",dog,samoyed,47.5
1,"[0.0, -1.0]",dog,corgi,26.0


In [69]:
len(table)

2

## Restore Table Versions ?

Errors is a common occurrence in AI. What's hard about errors in vector search is that oftentimes a bad vector doesn't cause a crash but just creates non-sensical answers. So to be able to rollback the state of the database is very important for debugging and reproducibility

So far we've accumulated 4 actions on the table:
1. creation of the table
2. added cats
3. added dogs
4. deleted cats

What if you realized that you should have deleted the dogs instead of the cats?

Here we can see the 4 versions that correspond to the 4 actions we've done

In [70]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 7, 4, 899414),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 7, 4, 997552),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 7, 5, 91048),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 14, 16, 436059),
  'metadata': {}}]

Please write code to restore the version still containing the whole dataset

In [71]:
table = db["cats_and_dogs"]

In [72]:
len(table)

2

In [78]:
# restore to version 3

table.restore(3)

In [79]:
table.to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5
2,"[0.0, 10.0]",dog,samoyed,47.5
3,"[0.0, -1.0]",dog,corgi,26.0


In [80]:
# delete the dogs instead

table.delete(" species = 'dog' ")

In [81]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 7, 4, 899414),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 7, 4, 997552),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 7, 5, 91048),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 14, 16, 436059),
  'metadata': {}},
 {'version': 5,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 16, 30, 935956),
  'metadata': {}},
 {'version': 6,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 16, 45, 932876),
  'metadata': {}},
 {'version': 7,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 16, 50, 554321),
  'metadata': {}},
 {'version': 8,
  'timestamp': datetime.datetime(2024, 1, 27, 21, 17, 10, 237523),
  'metadata': {}}]

In [82]:
table.to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


## Dropping a table

You can also choose to drop a table, which also completely removes the data.
Note that this operation is not reversible.

In [83]:
"cats_and_dogs" in db

True

Write code to irrevocably remove the table "cats_and_dogs" from the database

In [84]:
db.drop_table("cats_and_dogs")

How would you verify that the table has indeed been deleted?

In [85]:
table.name in db

False